# Tutorial 3: A First Model

After getting a basic understanding of the data in the previous tutorial, we are ready to build a first AI model.
In this notebook, we build and evaluate a simple model for identifying worm sections in nodule images.

By the end of this notebook you will have created a first submission that you can submit to put your team on the leaderboard!

**NOTE: This notebook will only work with a GPU instance. Make sure that you selected *ml.g4dn.xlarge* as instance type**

# Picking the right approach

The first challenge we have to address is how to even tackle the problem. What could be a good approach / algorithm? How are other people doing it?
[PapersWithCode](https://paperswithcode.com/sota) is an amazing resource for exactly these questions. It lists typical problem classes together with the current state-of-the-art and links to popular frameworks.
For our problem, [Object Detection](https://paperswithcode.com/task/object-detection) is the right category. If you follow the link you'll find a list of popular frameworks.

[MMDetection](https://github.com/open-mmlab/mmdetection) is one of the most widely used and the one we picked for this tutorial. Not only does it provide a great [tutorial](https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb), but also offers a variety of [different architectures and settings](https://github.com/open-mmlab/mmdetection#overview-of-benchmark-and-model-zoo) that you can play around with.

In this notebook, we will adapt the MMdetection [tutorial](https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb) to our problem and run it on a toy dataset.

For those familiar with deep learning architectures, MMDetection can handle data loading, preperation, model building, and training by altering a few common settings in the configuration file. Data preperation also includes various data augmentation techinques like rotation or flipping. The majority of the most common CV arcitectures are already build and available via configuration.

### *Further reading:*
Going into details of object detection is beyond the scope of this tutorial. If you'd like to know more, here are two recommendations to get started:
- [An introduction to R-CNNs and object detection in general](https://towardsdatascience.com/deep-dive-into-the-computer-vision-world-part-2-7a24efdb1a14)
- [An graphic explanation how convolutional neural networks work](https://towardsdatascience.com/gentle-dive-into-math-behind-convolutional-neural-networks-79a07dd44cf9)

# Setup
As in the other notebooks, we start by importing the relevant libraries and global settings.

In [4]:
import matplotlib.pyplot as plt  # Used for plotting
import mmcv  # Object detection framework
import os  # Interaction with the file system
import pandas as pd  # Home of the DataFrame construct, _the_ most important object for Data Science
import sys  # Python system library needed to load custom functions

from matplotlib.patches import Rectangle  # Allows drawing the bounding boxes of the worm sections
from mmcv import Config  # Loading and accessing MMDetection configuration files
from mmdet.apis import inference_detector, init_detector, train_detector, set_random_seed  # Part of the MMDetection framework
from mmdet.datasets import build_dataset  # Part of the MMDetection framework
from mmdet.models import build_detector  # Part of the MMDetection framework

from PIL import Image  # For loading image files
from tqdm import tqdm  # for timing a for loop

ImportError: /mmcv/mmcv/_ext.cpython-36m-x86_64-linux-gnu.so: undefined symbol: _ZN2at5sliceERKNS_6TensorElN3c108optionalIlEES5_l

In [ ]:
sys.path.append('../src')  # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.

In [ ]:
sys.path

In [6]:
from Dataset import OnchoDataset
from detection_util import create_predictions
from gdsc_score import get_leaderboard_score
from gdsc_util import download_directory, download_file, load_sections_df, set_up_logging, PROJECT_DIR
from PredictionEvaluator import PredictionEvaluator

set_up_logging()  # Sets up logging to console and the .log file
data_folder = str(PROJECT_DIR / 'data')

In [7]:
PROJECT_DIR
data_folder

'/root/data/gdsc5-tutorials-public/data'

In [8]:
# OnchoDataset

# Preparing the Training Dataset

Whenever you start working with a new framework you can expect issues with the setup.
To ensure that we can quickly try things out we create a small subset of our data.

In [9]:
section_df = load_sections_df(f'{data_folder}/gdsc_train.csv')
len(section_df)

65687

In [10]:
# preparing training and testing data 


Our complete dataset has more than 65.000 rows. Let's create two small dummy sets with 100 and 50 entries only.

In [ ]:
# TODO:: class imbalance handling 

In [2]:
dummy_train = section_df[1000:3000]
dummy_test = section_df[3000:3500]
dummy_train.to_csv(f'{data_folder}/dummy_train.csv', sep=';')
dummy_test.to_csv(f'{data_folder}/dummy_test.csv', sep=';')

NameError: name 'section_df' is not defined

We store the reduced sets under *../data*. Verify that you can find and open the files there. 
To make sure that everything works, let's load the files again.

In [12]:
dummy_train = load_sections_df(f'{data_folder}/dummy_train.csv')
dummy_train.head()

,file_name,study,staining,xmin,xmax,ymin,ymax,height,width
section_id,,,,,,,,,
105_A@4829-3641-5098-3824,105_A.jpg,Study_2,A,4829,5098,3641,3824,7137,8192
105_A@4888-2629-5122-2930,105_A.jpg,Study_2,A,4888,5122,2629,2930,7137,8192
105_A@4935-2054-5165-2250,105_A.jpg,Study_2,A,4935,5165,2054,2250,7137,8192
105_A@5003-1858-5295-2193,105_A.jpg,Study_2,A,5003,5295,1858,2193,7137,8192
105_A@5056-1716-5411-2057,105_A.jpg,Study_2,A,5056,5411,1716,2057,7137,8192


In [13]:
dummy_train.file_name.nunique()


28

Looks good! 

# Creating a MMDetection Configuration File

MMDetection relies on extensive configuration files. The usual process is to adapt an already existing configuration file and apply transfer learning.
To do this, we first need to download the configuration and weights files.

In [13]:
# Make a new folder in the data folder
# !mkdir ../data/checkpoints
# !mkdir ../data/checkpoints1
# !mkdir ../data/checkpoints/vfnet

# Install wget, a program for downloading files from the internet
# !apt update
# !apt install wget


#Vfnet config and weights file 
# vfnet with resnet 50 back bone 
!wget -c https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_1x_coco/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth -O ../data/checkpoints/vfnet/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth

# vfnet with resenet 50 back bone 
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth -o ../data/checkpoints/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth

# vfnet with r101 back bone 
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_r101_fpn_1x_coco/vfnet_r101_fpn_1x_coco_20201027pth-c831ece7.pth -O ../data/checkpoints/vfnet/vfnet_r101_fpn_1x_coco_20201027pth-c831ece7.pth
    
    
# Download the config and weights file
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth -O ../data/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

--2022-06-20 17:13:18--  https://download.openmmlab.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_1x_coco/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK

    The file is already fully retrieved; nothing to do.



In [14]:
from mmdet.apis import set_random_seed
from mmdet.apis import train_detector, init_detector
import torch 

The configuration file is part of the mmdetection package that is already installed on this image. Hence we can load our test configuration via

In [15]:
# Tutorial 3 
# faster RCNN

# cfg = Config.fromfile('/mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

In [15]:
# custom configuration for vfnet 
# cfg = Config.fromfile("/mmdetection/configs/vfnet/vfnet_r50_fpn_mstrain_2x_coco.py")
# cfg = Config.fromfile("/mmdetection/configs/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py")

# vfnet with r101 back bone 
# cfg = Config.fromfile("/mmdetection/configs/vfnet/vfnet_r101_fpn_1x_coco.py")
# ran with only 3 epochs

#vfnet with r50 back bone 
# submission(1-4)
cfg = Config.fromfile("/mmdetection/configs/vfnet/vfnet_r50_fpn_1x_coco.py")

The configuration is a dictionary-like object that stores our configuration settings. Is has [a lot](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html) of settings and with the right options, you can run the [really advanced models](https://github.com/open-mmlab/mmdetection/blob/master/docs/en/model_zoo.md).
Below we modify some of those settings to point the model to our train and test data.

We start by defining the structure and location of our data.

In [16]:
# Modify dataset type and path
cfg.dataset_type = 'OnchoDataset' #this is a custom data loader script we created for the GDSC you can view it in src/Dataset.py
cfg.data_root = data_folder

cfg.data.train.type = 'OnchoDataset'
cfg.data.train.data_root = data_folder      # path to the folder data
cfg.data.train.img_prefix = 'jpgs/'         # path from data_root to the images folder
cfg.data.train.ann_file = 'dummy_train.csv' # the file containing the train data labels

cfg.data.test.type = 'OnchoDataset'
cfg.data.test.data_root = data_folder
cfg.data.test.img_prefix = 'jpgs/'
cfg.data.test.ann_file = 'dummy_test.csv'

cfg.data.val.type = 'OnchoDataset'          # We will not use a separate validation data set in this tutorial, but we need to specify the values to overwrite the COCO defaults.
cfg.data.val.data_root = data_folder
cfg.data.val.img_prefix = 'jpgs/'
cfg.data.val.ann_file = 'dummy_test.csv'

We specify where to save the results and where the weights we will use were downloaded to. We also set the number of workers.

In [17]:
# We can still use the pre-trained Mask RCNN model though we do not need to use the mask branch
# cfg.load_from = f'{data_folder}/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

#vfnet model 
# cfg.load_from = f'{data_folder}/checkpoints1/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth'
# cfg.load_from = f'{data_folder}/checkpoints/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth'
# cfg.load_from = f'{data_folder}/checkpoints/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth'


# vfnet with r101 back bone 
# cfg.load_from = f'{data_folder}/checkpoints/vfnet/vfnet_r101_fpn_1x_coco_20201027pth-c831ece7.pth'


# vfnet with r50 back bone 
cfg.load_from = f'{data_folder}/checkpoints/vfnet/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth'



# Set up working dir to save files and logs.
cfg.work_dir = f'{data_folder}/vfnet_r50_exp5/'
# Ensure work_dir exists
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)  # not available in vfnet config

# cfg.data.samples_per_gpu = 3 # These numbers will change depending on the size of your model and GPU.
# cfg.data.samples_per_gpu = 1 # submission 1
cfg.data.samples_per_gpu = 2 #  

# cfg.data.workers_per_gpu = 1 # These values are what we have found to be best for this model and GPU
cfg.data.workers_per_gpu = 0 # These values are what we have found to be best for this model and GPU

cfg.device = 'cuda' # This defines that we will use a GPU

Next are the model settings. One thing we have to change is the number of classes to predict. Since we only have one class (worm section), we need to set this to one.
We also change the learning rate since we only have one GPU.

In [18]:
# modify number of classes of the model in box head
# cfg.model.roi_head.bbox_head.num_classes = 1  # a worm section is the only object we are detecting


#vfnet
cfg.model.bbox_head.num_classes = 1
cfg.classes = ("worm_section")




# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU and multiply by the number of GPU workers.
# cfg.optimizer.lr = 0.02 / 8 * cfg.data.workers_per_gpu
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

Finally, we specify how and when to evaluate the model and how long to train the model. We will use the [mAP](https://blog.paperspace.com/mean-average-precision/#:~:text=To%20evaluate%20object%20detection%20models,model%20is%20in%20its%20detections.) metric to evaluate how the training is going.

For our test run, we will only train for three epochs. For a real training run you'll want to set this number higher.
After each epoch, the current weights will be saved in the *work_dir* folder we set above. Also the model will be evaluated on the test data set.
This allows us to check if we should continue training or stop.

In [19]:
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 3
# How long do we want to train
cfg.runner.max_epochs = 10

In [20]:
# cfg.auto_scale_lr.base_batch_size = 16 # submission 1 (3 epochs)
# cfg.auto_scale_lr.base_batch_size = 8 # submission 2
# cfg.auto_scale_lr.base_batch_size = 8 # submission 3
# cfg.auto_scale_lr.base_batch_size = 8 # submission 4
# cfg.auto_scale_lr.base_batch_size = 16 # submission 5 

cfg.auto_scale_lr.base_batch_size = 32 # submission 6 


# cfg.optimizer.lr = 0.02/8*2 #submission 1 > (3 epochs)
# cfg.optimizer.lr = 0.02/8 #submission 2   > 2.5 learning rate 
# cfg.optimizer.lr = 0.02/8*0 #submission 3 > 0 learning rate 
# cfg.optimizer.lr = 0.001 #submission 4    > 1.000e-03 learning rate 
cfg.optimizer.lr = 0.002  # submission 5 , 6  



In [21]:
0.02/8

0.0025

In [22]:
cfg

Config (path: /mmdetection/configs/vfnet/vfnet_r50_fpn_1x_coco.py): {'dataset_type': 'OnchoDataset', 'data_root': '/root/data/gdsc5-tutorials-public/data', 'img_norm_cfg': {'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, 'train_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True}, {'type': 'Resize', 'img_scale': (1333, 800), 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels']}], 'test_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': [{'type': 'Resize', 'keep_ratio': True}, {'type': 'RandomFlip'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb

# Training of the first Model

We're now ready to train a first simple model! The first step is to prepare the dataset.

In [23]:
datasets = [build_dataset(cfg.data.train)]

datasets

2022-06-20 20:46:23,594 - Dataset - INFO - Building Dataset
2022-06-20 20:46:23,594 - Dataset - INFO - Building Dataset


/mmdetection/mmdet/datasets/custom.py:180: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


[
 OnchoDataset Train dataset with number of images 28, and instance counts: 
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | category    | count | category | count | category | count | category | count | category | count |
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 |             |       |          |       |          |       |          |       |          |       |
 | 0 [section] | 2000  |          |       |          |       |          |       |          |       |
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+]

The dataset contains exactly 100 worm sections, one for each row on the *dummy_train* DataFrame. We also get a warning about filtering empty ground truth images which we can ignore since we don't have those.

Next, we initialize the model according to the configuration. *train_cfg* determins on which data to train the model, *test_cfg* on which data to test the model.

In [24]:
datasets[0].CLASSES

['section']

In [25]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES  # Add an attribute for visualization convenience


In [26]:
train_detector

<function mmdet.apis.train.train_detector(model, dataset, cfg, distributed=False, validate=False, timestamp=None, meta=None)>

And finally, we can train the model. The log output helps us understand how well the model is doing. 
Since we set the *evaluation.interval* to *1*, the model will be evaluated after every epoch. In general you want to continue training as long as the *mAP* score increases.

In [27]:
train_detector(model, datasets, cfg, validate=True)

2022-06-20 20:46:34,321 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-06-20 20:46:34,321 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-06-20 20:46:34,345 - Dataset - INFO - Building Dataset
2022-06-20 20:46:34,345 - Dataset - INFO - Building Dataset
2022-06-20 20:46:34,379 - mmdet - INFO - load checkpoint from local path: /root/data/gdsc5-tutorials-public/data/checkpoints/vfnet/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth
2022-06-20 20:46:34,379 - mmdet - INFO - load checkpoint from local path: /root/data/gdsc5-tutorials-public/data/checkpoints/vfnet/vfnet_r50_fpn_1x_coco_20201027-38db6f58.pth
2022-06-20 20:46:34,528 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([1, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a pa

The log starts with a warning about a size mismatch that we can ignore. It is due to the fact that the weights we load belong to a model that was trained on the COCO dataset which has 80 different classes whereas we only have one class to detect.

Since the *mAP* keep increasing it would make sense to continue training the model for longer. 
But before we do that let's make sure that the rest of the process works.

**Note**: 
- **If you get an error, try rerunning the notebook starting with the loading of the config.** 
- **If you get the error *RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 5-dimensional input of size [1, 3, 3, 928, 768] instead* change the instance to one with a GPU.** 



# Running the Model

After training the model we can now apply it on a new image and look at the predictions. 
For this, we first load the image and initalize the model from a stored checkpoint.

In [31]:
example_image = '1_D.jpg'
img =  mmcv.imread(f'{data_folder}/jpgs/{example_image}')

In [29]:
# checkpoint = f'{cfg.work_dir}epoch_3.pth' # Select one of the model checkpoints to load in
# checkpoint = f'{cfg.work_dir}epoch_12.pth' # 
# checkpoint = f'{cfg.work_dir}epoch_36.pth' #
# checkpoint = f'{cfg.work_dir}epoch_8.pth' # submission 5
checkpoint = f'{cfg.work_dir}epoch_10.pth' # submission 6

model = init_detector(cfg, checkpoint, device='cuda')

load checkpoint from local path: /root/data/gdsc5-tutorials-public/data/vfnet_r50_exp5/epoch_10.pth


Run the inference

In [32]:
detections = inference_detector(model, img)
print(len(detections))  
detections[0][:2]  # Show the first two entries

1


array([[4.3058022e+03, 3.7138210e+03, 5.1693896e+03, 4.5976689e+03,
        7.5891238e-01],
       [2.8863604e+03, 8.3972986e+02, 3.5336499e+03, 1.5224211e+03,
        7.5781757e-01]], dtype=float32)

*detections* is a list of all the detected worm sections. Every detection is a 5-tuple of the form *(xmin, ymin, xmax, ymax, score)*, i.e. the location of the bounding box followed by the confidence.

**Exercise:**
- Plot the predicted worm boxes. You may use the code provided in tutorial 2.
- Load the model weights of a different checkpoint, e.g. after the first epoch. What are the differences?

# Evaluating the model

To get an idea how well we're doing we can run the same evaluation function that will be used for the leaderboard on our example file.
For this we first need to convert to model output into the format that is readable by our scoring function.

In [33]:
columns = ['section_id', 'file_name', 'xmin', 'xmax', 'ymin', 'ymax', 'detection_score']
df_results = pd.DataFrame(columns=columns)

predictions = []

# Loop over all detected boxes and convert the information into a dictionary
for box in detections[0]:
    xmin, ymin, xmax, ymax, score = box
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)  # Convert predicted coordinates to integer values
    box_dict = dict(
        section_id=f'{example_image}@{xmin}-{xmax}-{ymin}-{ymax}',
        file_name=example_image,
        xmin=xmin,
        ymin=ymin,
        xmax=xmax,
        ymax=ymax,
        detection_score=score
    )
    predictions.append(box_dict)

# Convert the dictionary into a dataframe with section_id as index
prediction_df = pd.DataFrame(predictions)
prediction_df.set_index('section_id', inplace=True)
prediction_df.head()

,file_name,xmin,ymin,xmax,ymax,detection_score
section_id,,,,,,
1_D.jpg@4305-5169-3713-4597,1_D.jpg,4305,3713,5169,4597,0.758912
1_D.jpg@2886-3533-839-1522,1_D.jpg,2886,839,3533,1522,0.757818
1_D.jpg@5347-5873-2714-3309,1_D.jpg,5347,2714,5873,3309,0.754759
1_D.jpg@3870-4640-2477-3671,1_D.jpg,3870,2477,4640,3671,0.739739
1_D.jpg@4612-5161-3199-3624,1_D.jpg,4612,3199,5161,3624,0.735658


Next, we restrict our dataset to only the file for which we made predictions (*1_D.jpg*) and run the score function

In [34]:
ground_truth = section_df.loc[section_df.file_name==example_image]

In [35]:
evaluator = PredictionEvaluator(ground_truth)
thresholds = [0.5, 0.6, 0.7]

For the leaderboard, the predicted worm section boxes are compared to the actual worm section boxes. We compute the [Intersection over Union (IOU)](https://towardsdatascience.com/iou-a-better-detection-evaluation-metric-45a511185be1) between the boxes. If the value is greater than a threshold (e.g. 0.5) the prediction counts as a match.

In [36]:
get_leaderboard_score(prediction_df, thresholds, evaluator)

2022-06-20 21:02:40,355 - gdsc_score - INFO - Computing results for threshold: 0.5
2022-06-20 21:02:40,356 - PredictionEvaluator - INFO - Matching sections


100%|██████████| 1/1 [00:00<00:00, 24.63it/s]

2022-06-20 21:02:40,400 - PredictionEvaluator - INFO - Merging matched sections
2022-06-20 21:02:40,410 - PredictionEvaluator - INFO - Done matching sections
2022-06-20 21:02:40,410 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-20 21:02:40,411 - PredictionEvaluator - INFO - Computing overall scores
2022-06-20 21:02:40,413 - gdsc_score - INFO - Computing results for threshold: 0.6
2022-06-20 21:02:40,414 - PredictionEvaluator - INFO - Matching sections



100%|██████████| 1/1 [00:00<00:00, 25.15it/s]

2022-06-20 21:02:40,457 - PredictionEvaluator - INFO - Merging matched sections
2022-06-20 21:02:40,465 - PredictionEvaluator - INFO - Done matching sections
2022-06-20 21:02:40,466 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-20 21:02:40,467 - PredictionEvaluator - INFO - Computing overall scores
2022-06-20 21:02:40,469 - gdsc_score - INFO - Computing results for threshold: 0.7
2022-06-20 21:02:40,469 - PredictionEvaluator - INFO - Matching sections



100%|██████████| 1/1 [00:00<00:00, 25.38it/s]

2022-06-20 21:02:40,512 - PredictionEvaluator - INFO - Merging matched sections
2022-06-20 21:02:40,521 - PredictionEvaluator - INFO - Done matching sections
2022-06-20 21:02:40,521 - PredictionEvaluator - INFO - Evaluating predictions
2022-06-20 21:02:40,522 - PredictionEvaluator - INFO - Computing overall scores


{'detection_acc@iou0.5': 35.92,
 'detection_tp@iou0.5': 37,
 'detection_fp@iou0.5': 63,
 'detection_fn@iou0.5': 3,
 'detection_acc@iou0.6': 33.33,
 'detection_tp@iou0.6': 35,
 'detection_fp@iou0.6': 65,
 'detection_fn@iou0.6': 5,
 'detection_acc@iou0.7': 32.08,
 'detection_tp@iou0.7': 34,
 'detection_fp@iou0.7': 66,
 'detection_fn@iou0.7': 6,
 'score': 101.33}

The function *get_leaderboard_score* gives us a lot of information that we can use to debug our model. 
- *score* is the score that will be shown in the leaderboard. It is the sum of the individual detection_acc@X values.
- Additionally, for each IOU threshold the *accuracy, true positives, false positives* and *false negatives* are computed.

**Exercise:**
- Which type of error is the most common? How could you combat this?
- The output shows true positives (tp), false positives (fp), false negatives (fn) but no true negatives. Why is that?
- Evaluate the model on the *dummy_test* dataset we created previously.

# A first submission

Now, we have all the pieces in place to create our first submission. We will predict the worm sections for all files and store them in a DataFrame.
We will need to provide a prediction for all the files that are listed in *test_files.csv*. Let's load the file first.

In [37]:
files = pd.read_csv(f'{data_folder}/test_files.csv', sep=';', header=None)
file_names = files[0].values
file_names

array(['100_D.jpg', '100_C.jpg', '100_B.jpg', '100_AA.jpg', '100_A.jpg',
       '101_DD.jpg', '101_C.jpg', '101_B.jpg', '101_AA.jpg', '101_A.jpg',
       '86_D.jpg', '86_C.jpg', '86_B.jpg', '86_AA.jpg', '86_A.jpg',
       '88_D.jpg', '88_C.jpg', '88_B.jpg', '88_A.jpg', '89_D.jpg',
       '89_C.jpg', '89_B.jpg', '89_AA.jpg', '89_A.jpg', '90_D.jpg',
       '90_C.jpg', '90_B.jpg', '90_AA.jpg', '90_A.jpg', '91_D.jpg',
       '91_C.jpg', '91_B.jpg', '91_AA.jpg', '91_A.jpg', '92_D.jpg',
       '92_C.jpg', '92_B.jpg', '92_AA.jpg', '92_A.jpg', '93_D.jpg',
       '93_C.jpg', '93_B.jpg', '93_AA.jpg', '93_A.jpg', '94_D.jpg',
       '94_C.jpg', '94_B.jpg', '94_AA.jpg', '94_A.jpg', '95_D.jpg',
       '95_C.jpg', '95_B.jpg', '95_AA.jpg', '95_A.jpg', '96_D.jpg',
       '96_C.jpg', '96_B.jpg', '96_AA.jpg', '96_A.jpg', '97_D.jpg',
       '97_C.jpg', '97_B.jpg', '97_A.jpg', '98_D.jpg', '98_C.jpg',
       '98_B.jpg', '98_AA.jpg', '98_A.jpg', '99_D.jpg', '99_C.jpg',
       '99_B.jpg', '99_AA.jpg', '99_A.j

For simplicity, we created a function that runs the above code on all file names and returns a dataframe with the predictions. This will take around five minutes.

In [38]:
prediction_df = create_predictions(file_names, cfg, checkpoint, device='cuda')

load checkpoint from local path: /root/data/gdsc5-tutorials-public/data/vfnet_r50_exp5/epoch_10.pth
2022-06-20 21:03:05,185 - detection_util - INFO - Creating predictions for 73 files


  0%|          | 0/73 [00:00<?, ?it/s]

2022-06-20 21:03:05,188 - detection_util - INFO - Processing file: 100_D.jpg


  1%|▏         | 1/73 [00:00<01:08,  1.05it/s]

2022-06-20 21:03:06,141 - detection_util - INFO - Processing file: 100_C.jpg


...

 97%|█████████▋| 71/73 [01:13<00:01,  1.08it/s]

2022-06-20 21:04:19,051 - detection_util - INFO - Processing file: 99_AA.jpg


 99%|█████████▊| 72/73 [01:14<00:00,  1.15it/s]

2022-06-20 21:04:19,775 - detection_util - INFO - Processing file: 99_A.jpg


100%|██████████| 73/73 [01:15<00:00,  1.04s/it]


In [78]:
prediction_df.head()

,file_name,xmin,ymin,xmax,ymax,detection_score
section_id,,,,,,
100_D.jpg@1035-1212-1140-1428,100_D.jpg,1035,1140,1212,1428,0.645738
100_D.jpg@382-692-1289-1691,100_D.jpg,382,1289,692,1691,0.641257
100_D.jpg@1258-1469-1187-1400,100_D.jpg,1258,1187,1469,1400,0.629040
100_D.jpg@1727-1962-4362-4603,100_D.jpg,1727,4362,1962,4603,0.624631
100_D.jpg@1881-2061-2587-2841,100_D.jpg,1881,2587,2061,2841,0.609023


Looks good! All that's left is to save the results to a csv and upload it on the [GDSC website](https://gdsc.ce.capgemini.com/).

In [39]:
# prediction_df.to_csv(f'{data_folder}/results_tutorial3.csv', sep=';')
# prediction_df.to_csv(f'{data_folder}/themav_sub1.csv', sep=';') # submission 1 
# prediction_df.to_csv(f'{data_folder}/vfnet_r50_sub2.csv', sep=';') # submission 2 
# prediction_df.to_csv(f'{data_folder}/vfnet_r50_sub3.csv', sep=';') # submission 3 wrong submission lr rate not adjusted 
# prediction_df.to_csv(f'{data_folder}/vfnet_r50_sub4.csv', sep=';') # submission 4 
# prediction_df.to_csv(f'{data_folder}/vfnet_r50_sub5.csv', sep=';') # submission 5 
prediction_df.to_csv(f'{data_folder}/vfnet_r50_sub6.csv', sep=';') # submission 6 


This submission should lead to a score of around 28. 

**Exercise:**
- The submission score is a lot less than the score of 70.07 we computed above. What are potential reasons for this?

# Summary

This notebook covered a LOT of different topics. We covered how to 
- Train a basic object detection model
- Use a trained model to create predictions on a nodule image
- How to evaluate the predictions
- How to create a first submission

all done on a dummy dataset. With this, we have the technical foundations to creating a good submission!

In the next tutorial, you will learn how to run and evaluate the model from this notebook on the complete dataset as a Sagemaker training job.

**REMINDER: Remember to shut down the *ml.g4dn.xlarge* instance when you aren't using it.**

In [5]:
import pandas as pd

In [2]:
! pwd
%cd ../yolov5_river

/root/gdsc5-tutorials-public/notebooks
/root/gdsc5-tutorials-public/yolov5_river


In [3]:
! ls

data  first_yolo_pred.csv  yolov5


In [6]:
df = pd.read_csv("first_yolo_pred.csv")

In [8]:
%cd ../data/

/root/gdsc5-tutorials-public/data


In [9]:
! ls

checkpoints	dummy_train.csv  jpgs		 themav_sub1.csv  txts
dummy_test.csv	gdsc_train.csv	 test_files.csv  tutorial_exps


In [11]:
df2 = pd.read_csv("themav_sub1.csv")

In [14]:
df2.shape , df.shape

((6773, 1), (3375, 1))